### Data - Local


In [ ]:
import pandas as pd
import numpy as np

application_train = pd.read_csv(
    ".tmp/home-credit-default-risk/application_train.csv", low_memory=False
)
bureau = pd.read_csv(
    ".tmp/home-credit-default-risk/bureau.csv",
    low_memory=False,
)
previous_application = pd.read_csv(
    ".tmp/home-credit-default-risk/previous_application.csv", low_memory=False
)
credit_card_balance = pd.read_csv(
    ".tmp/home-credit-default-risk/credit_card_balance.csv", low_memory=False
)

### Data - GDrive


In [ ]:
# from google.colab import drive

# drive.mount("/content/drive")
# import pandas as pd
# import numpy as np
# FILE_PATH = (
#     "/content/drive/MyDrive/Colab Notebooks/middle_project/data/bureau.csv의 사본"
# )

# bureau = pd.read_csv(FILE_PATH)
# FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/middle_project/data/previous_application.csv의 사본"

# previous_application = pd.read_csv(FILE_PATH)
# FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/middle_project/data/credit_card_balance.csv의 사본"

# credit_card_balance = pd.read_csv(FILE_PATH)
# FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/middle_project/data/application_train.csv의 사본"

# application_train = pd.read_csv(FILE_PATH)

# previous_application


## bureau와 중복된 이전대출 제거 (previous.csv에서 제거 예정)


In [ ]:
import os

contract_type_mapping = {
    "Consumer credit": "Consumer loans",
    "Cash loan (non-earmarked)": "Cash loans",
}
merged_df = pd.merge(
    bureau,
    previous_application,
    on="SK_ID_CURR",
    how="inner",
    suffixes=("_bureau", "_previous"),
)
merged_df["CREDIT_TYPE_mapped"] = merged_df["CREDIT_TYPE"].map(contract_type_mapping)

same_loan_df = merged_df[
    (merged_df["AMT_CREDIT_SUM"] == merged_df["AMT_CREDIT"])
    & (merged_df["DAYS_CREDIT"] == merged_df["DAYS_DECISION"])
    & (merged_df["CREDIT_TYPE_mapped"] == merged_df["NAME_CONTRACT_TYPE"])
]

previous_application = previous_application[
    ~previous_application["SK_ID_PREV"].isin(same_loan_df["SK_ID_PREV"])
]
print(previous_application.shape)  # 1670214 -> (1664865, 38)

## 파생변수


#### LOAN_STATUS


In [ ]:
def determine_loan_status(row):
    # 대출이 취소된 경우
    if row["NAME_CONTRACT_STATUS"] in ["Canceled", "Refused", "Unused offer"]:
        return 3

    # 대출이 승인되었지만, 상환 시작이 안된 경우
    if pd.isna(row["DAYS_FIRST_DUE"]) or row["DAYS_FIRST_DUE"] > 0:
        return 3

    # 대출이 승인되었고, 대출 종료 시점(DAYS_TERMINATION)이 존재하며 상환 완료된 경우
    if (
        row["DAYS_TERMINATION"] < 0
        and pd.notna(row["DAYS_LAST_DUE"])
        and row["DAYS_LAST_DUE"] < row["DAYS_TERMINATION"]
    ):
        return 0

    # 대출이 승인되었고, 현재 상환 중인 경우 (상환 기한이 남아 있고, 연체되지 않은 상태)
    if (
        pd.notna(row["DAYS_FIRST_DUE"])
        and row["DAYS_LAST_DUE"] > 0
        and row["DAYS_TERMINATION"] > 0
    ):
        return 2

    # 연체된 경우 (상환 기한이 지나거나, 종료 시점이 지났음에도 상환되지 않은 경우)
    if (pd.notna(row["DAYS_LAST_DUE"]) and row["DAYS_LAST_DUE"] < 0) or (
        pd.notna(row["DAYS_TERMINATION"])
        and row["DAYS_TERMINATION"] < 0
        and row["DAYS_LAST_DUE"] >= row["DAYS_TERMINATION"]
    ):
        return 1

    # 나머지 경우는 '확인 필요'로 표시 (모든 조건에 해당하지 않는 경우)
    return 4


# 새 파생 변수 생성
previous_application["LOAN_STATUS"] = previous_application.apply(
    determine_loan_status, axis=1
)

# 결과 확인
display(
    previous_application[
        [
            "SK_ID_CURR",
            "NAME_CONTRACT_STATUS",
            "DAYS_TERMINATION",
            "DAYS_LAST_DUE",
            "LOAN_STATUS",
        ]
    ]
)

In [ ]:
previous_application["LOAN_STATUS"].value_counts()

#### IS_REVOLVING_LOAN


In [ ]:
previous_application["IS_REVOLVING_LOAN"] = previous_application[
    "NAME_CONTRACT_TYPE"
].apply(lambda x: 1 if x == "Revolving loans" else 0)

## previous_application table 완성본


In [ ]:
# NFLAG_INSURED_ON_APPROVALE
previous_application["NFLAG_INSURED_ON_APPROVAL_COUNT"] = previous_application.groupby(
    "SK_ID_CURR"
)["NFLAG_INSURED_ON_APPROVAL"].transform("count")
previous_application["NFLAG_INSURED_ON_APPROVAL_SUM"] = previous_application.groupby(
    "SK_ID_CURR"
)["NFLAG_INSURED_ON_APPROVAL"].transform("sum")
previous_application["NFLAG_INSURED_ON_APPROVAL_RATIO"] = (
    previous_application["NFLAG_INSURED_ON_APPROVAL_SUM"]
    / previous_application["NFLAG_INSURED_ON_APPROVAL_COUNT"]
)

# AMT_APPLICATION
previous_application["AMT_APPLICATION_MAX"] = previous_application.groupby(
    "SK_ID_CURR"
)["AMT_APPLICATION"].transform("max")
previous_application["AMT_APPLICATION_SUM"] = previous_application.groupby(
    "SK_ID_CURR"
)["AMT_APPLICATION"].transform("sum")
previous_application["AMT_APPLICATION_MEAN"] = previous_application.groupby(
    "SK_ID_CURR"
)["AMT_APPLICATION"].transform("mean")

# RATE_DOWN_PAYMENT
previous_application["RATE_DOWN_PAYMENT_MEAN"] = previous_application.groupby(
    "SK_ID_CURR"
)["RATE_DOWN_PAYMENT"].transform("mean")

# # RATE_INTEREST_PRIMARY
# previous_application['RATE_INTEREST_PRIMARY_MEAN'] = previous_application.groupby('SK_ID_CURR')['RATE_INTEREST_PRIMARY'].transform('mean')

# # RATE_INTEREST_PRIVILEGED
# previous_application['RATE_INTEREST_PRIVILEGED_MEAN'] = previous_application.groupby('SK_ID_CURR')['RATE_INTEREST_PRIVILEGED'].transform('mean')

tri_previous_application = previous_application[
    [
        "SK_ID_CURR",
        "NFLAG_INSURED_ON_APPROVAL_COUNT",
        "NFLAG_INSURED_ON_APPROVAL_SUM",
        "NFLAG_INSURED_ON_APPROVAL_RATIO",
        "AMT_APPLICATION_MAX",
        "AMT_APPLICATION_SUM",
        "AMT_APPLICATION_MEAN",
        "RATE_DOWN_PAYMENT_MEAN",
        "IS_REVOLVING_LOAN",
    ]
].copy()
tri_previous_application.drop_duplicates(subset="SK_ID_CURR", inplace=True)
tri_previous_application.dropna(
    inplace=True
)  # NaN값 대체 불가능한 파생변수 제거완료, 원본에서 보험가입여부가 NaN값이 많아서 drop

In [ ]:
tri_previous_application.info()

In [ ]:
import os.path as path

DATA_PATH = ".tmp/preprocessing"
if not path.isdir(DATA_PATH):
    os.makedirs(DATA_PATH, exist_ok=True)

tri_previous_application.to_csv(
    path.join(DATA_PATH, "tri_previous_application.csv"), index=False
)

# bureau


### 파생변수


#### Loan_status

-   완납 0, 연체1, 상환 중2, 취소 3, Unknown 4 => 삭제 예정 XNA or NaN


In [ ]:
def determine_loan_status(row):
    if row["CREDIT_ACTIVE"] == "Closed" and row["AMT_CREDIT_SUM_DEBT"] == 0:
        return 0
    elif (
        row["CREDIT_ACTIVE"] in ["Active", "Bad debt"] and row["CREDIT_DAY_OVERDUE"] > 0
    ):
        return 1
    elif (
        row["CREDIT_ACTIVE"] == "Active"
        and row["CREDIT_DAY_OVERDUE"] == 0
        and row["AMT_CREDIT_SUM_DEBT"] > 0
    ):
        if row["DAYS_CREDIT_ENDDATE"] < 0:  # 대출이 이미 종료된 날짜
            return 1
        else:
            return 2
    elif row["CREDIT_ACTIVE"] == "Sold":
        return 3
    else:
        return 4


bureau["LOAN_STATUS"] = bureau.apply(determine_loan_status, axis=1)

#### CREDIT_DAY_OVERDUE


In [ ]:
# breau에서 'CREDIT_DAY_OVERDUE'를 카테고리화 하기(0 : Overdue_0, 1-29일 연체 : Overdue_1, 30-59일 연체 : Overdue_2, 60-89일 연체 : Overdue_3, 90-119일 연체 : Overdue_4, 120일 이상 연체 : Overdue_5로 카테고리화)
def classify_overdue_days(days):
    if days == 0:
        return "Overdue_0"
    elif 1 <= days <= 29:
        return "Overdue_1"
    elif 30 <= days <= 59:
        return "Overdue_2"
    elif 60 <= days <= 89:
        return "Overdue_3"
    elif 90 <= days <= 119:
        return "Overdue_4"
    elif days >= 120:
        return "Overdue_5"
    else:
        return "Unknown"  # 만약 잘못된 데이터가 있을 경우


# classify_overdue_days 함수를 CREDIT_DAY_OVERDUE 컬럼에 적용
bureau["OVERDUE_CATEGORY"] = bureau["CREDIT_DAY_OVERDUE"].apply(classify_overdue_days)
# 결과 출력
bureau["OVERDUE_CATEGORY"].value_counts()

bureau.drop("CREDIT_DAY_OVERDUE", axis=1)
bureau.rename(columns={"OVERDUE_CATEGORY": "CREDIT_DAY_OVERDUE"}, inplace=True)

## bureau table 완성본


In [ ]:
# AMT_CREDIT_MAX_OVERDUE의 MAX, SUM 컬럼값 생성
bureau["AMT_CREDIT_MAX_OVERDUE_MAX"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_MAX_OVERDUE"
].transform("max")
bureau["AMT_CREDIT_MAX_OVERDUE_SUM"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_MAX_OVERDUE"
].transform("sum")

# CNT_CREDIT_PROLONG의 MAX, SUM, MEAN 컬럼값 생성
bureau["CNT_CREDIT_PROLONG_MAX"] = bureau.groupby("SK_ID_CURR")[
    "CNT_CREDIT_PROLONG"
].transform("max")
bureau["CNT_CREDIT_PROLONG_SUM"] = bureau.groupby("SK_ID_CURR")[
    "CNT_CREDIT_PROLONG"
].transform("sum")
bureau["CNT_CREDIT_PROLONG_MEAN"] = bureau.groupby("SK_ID_CURR")[
    "CNT_CREDIT_PROLONG"
].transform("mean")

# CREDIT_DAY_OVERDUE의 연체 카테고리 count 컬럼값 생성
overdue_counts = (
    bureau.groupby("SK_ID_CURR")["CREDIT_DAY_OVERDUE"]
    .value_counts()
    .unstack(fill_value=0)
)
overdue_counts.columns = [f"COUNT_{col}" for col in overdue_counts.columns]
bureau = bureau.merge(overdue_counts, on="SK_ID_CURR", how="left")

# AMT_CREDIT_SUM_DEBT의 MAX, SUM, MEAN 컬럼값 생성
bureau["AMT_CREDIT_SUM_DEBT_MAX"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_SUM_DEBT"
].transform("max")
bureau["AMT_CREDIT_SUM_DEBT_SUM"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_SUM_DEBT"
].transform("sum")
bureau["AMT_CREDIT_SUM_DEBT_MEAN"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_SUM_DEBT"
].transform("mean")

# AMT_CREDIT_SUM_LIMIT의 MEAN
bureau["AMT_CREDIT_SUM_LIMIT_MEAN"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_SUM_LIMIT"
].transform("mean")

# AMT_CREDIT_SUM_OVERDUE의 MAX, SUM, MEAN
bureau["AMT_CREDIT_SUM_OVERDUE_MAX"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_SUM_OVERDUE"
].transform("max")
bureau["AMT_CREDIT_SUM_OVERDUE_SUM"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_SUM_OVERDUE"
].transform("sum")
bureau["AMT_CREDIT_SUM_OVERDUE_MEAN"] = bureau.groupby("SK_ID_CURR")[
    "AMT_CREDIT_SUM_OVERDUE"
].transform("mean")

In [ ]:
# SK_ID_CURR 기준으로 중복값 제거 후, 특정 컬럼만 뽑아서 bureau_data_sample 생성
tri_bureau = bureau[
    [
        "SK_ID_CURR",
        "AMT_CREDIT_MAX_OVERDUE_MAX",
        "AMT_CREDIT_MAX_OVERDUE_SUM",
        "CNT_CREDIT_PROLONG_MAX",
        "CNT_CREDIT_PROLONG_SUM",
        "CNT_CREDIT_PROLONG_MEAN",
        "AMT_CREDIT_SUM_DEBT_MAX",
        "AMT_CREDIT_SUM_DEBT_SUM",
        "AMT_CREDIT_SUM_DEBT_MEAN",
        "AMT_CREDIT_SUM_LIMIT_MEAN",
        "AMT_CREDIT_SUM_OVERDUE_MAX",
        "AMT_CREDIT_SUM_OVERDUE_SUM",
        "AMT_CREDIT_SUM_OVERDUE_MEAN",
        "COUNT_Overdue_0",
        "COUNT_Overdue_1",
        "COUNT_Overdue_2",
        "COUNT_Overdue_3",
        "COUNT_Overdue_4",
        "COUNT_Overdue_5",
    ]
].copy()
tri_bureau.drop_duplicates(subset="SK_ID_CURR", inplace=True)
tri_bureau.fillna(
    0, inplace=True
)  # 전부 파생변수에서 NaN값 하지만 완납 or 취소인 경우에 부채에서 NaN을 발생시켜서 0으로 대체

# 결측치 확인
tri_bureau.info()

In [ ]:
tri_bureau.to_csv(path.join(DATA_PATH, "tri_bureau.csv"), index=False)

# credit_card_balance


### 파생변수


#### is_overdue


In [ ]:
# 연체 여부를 판단하는 칼럼
credit_card_balance["is_overdue"] = credit_card_balance["SK_DPD"] > 0

#### credit_limit_usage


In [ ]:
credit_card_balance["credit_limit_usage"] = (
    credit_card_balance["AMT_BALANCE"] / credit_card_balance["AMT_CREDIT_LIMIT_ACTUAL"]
).replace([np.inf, -np.inf], np.nan)

### credit_card_balance 완성본


In [ ]:
credit_card_balance["avg_credit_limit_usage"] = credit_card_balance.groupby(
    "SK_ID_CURR"
)["credit_limit_usage"].transform("mean")
credit_card_balance["total_overdue"] = credit_card_balance.groupby("SK_ID_CURR")[
    "is_overdue"
].transform("sum")
credit_card_balance["CNT_DRAWINGS_CURRENT_SUM"] = credit_card_balance.groupby(
    "SK_ID_CURR"
)["CNT_DRAWINGS_CURRENT"].transform("sum")

# SK_DPD의 컬럼값 생성
credit_card_balance["SK_DPD_MAX"] = credit_card_balance.groupby("SK_ID_CURR")[
    "SK_DPD"
].transform("max")
credit_card_balance["SK_DPD_MEAN"] = credit_card_balance.groupby("SK_ID_CURR")[
    "SK_DPD"
].transform("mean")

tri_credit_card_balance = credit_card_balance[
    [
        "SK_ID_CURR",
        "CNT_DRAWINGS_CURRENT_SUM",
        "SK_DPD_MAX",
        "SK_DPD_MEAN",
        "avg_credit_limit_usage",
        "total_overdue",
    ]
].copy()
tri_credit_card_balance.drop_duplicates(subset="SK_ID_CURR", inplace=True)

# na 제거
tri_credit_card_balance.dropna(
    inplace=True
)  # avg_credit_limit_usage 에서만 NaN값 존재 => 비율이라 0으로 하면 큰일나서 걍 drop하겠음

# 결측치 확인
tri_credit_card_balance.info()

In [ ]:
tri_credit_card_balance.to_csv(
    path.join(DATA_PATH, "tri_credit_card_balance.csv"), index=False
)

# bureau + previous_application


In [ ]:
# bureau
bureau_selected = bureau[
    ["SK_ID_CURR", "AMT_ANNUITY", "AMT_CREDIT_SUM", "LOAN_STATUS", "SK_ID_BUREAU"]
].copy()
bureau_selected.rename(
    columns={"AMT_CREDIT_SUM": "AMT_CREDIT", "SK_ID_BUREAU": "SK_ID"}, inplace=True
)

# previous_application
previous_selected = previous_application[
    ["SK_ID_CURR", "AMT_ANNUITY", "AMT_CREDIT", "LOAN_STATUS", "SK_ID_PREV"]
].copy()
previous_selected.rename(columns={"SK_ID_PREV": "SK_ID"}, inplace=True)

combined_df = pd.concat([bureau_selected, previous_selected], ignore_index=True)

In [ ]:
# CNT_CREDIT_PROLONG의 MAX, SUM, MEAN 컬럼값 생성
combined_df["AMT_ANNUITY_MAX"] = combined_df.groupby("SK_ID_CURR")[
    "AMT_ANNUITY"
].transform("max")
combined_df["AMT_ANNUITY_SUM"] = combined_df.groupby("SK_ID_CURR")[
    "AMT_ANNUITY"
].transform("sum")
combined_df["AMT_ANNUITY_MEAN"] = combined_df.groupby("SK_ID_CURR")[
    "AMT_ANNUITY"
].transform("mean")

combined_df["AMT_CREDIT_MAX"] = combined_df.groupby("SK_ID_CURR")[
    "AMT_CREDIT"
].transform("max")
combined_df["AMT_CREDIT_SUM"] = combined_df.groupby("SK_ID_CURR")[
    "AMT_CREDIT"
].transform("sum")
combined_df["AMT_CREDIT_MEAN"] = combined_df.groupby("SK_ID_CURR")[
    "AMT_CREDIT"
].transform("mean")
combined_df["AMT_CREDIT_MEDIAN"] = combined_df.groupby("SK_ID_CURR")[
    "AMT_CREDIT"
].transform("median")

combined_df["IS_OVERDUE"] = combined_df["LOAN_STATUS"].apply(
    lambda x: 1 if x == 1 else 0
)
combined_df["IS_PAIDOFF"] = combined_df["LOAN_STATUS"].apply(
    lambda x: 1 if x == 0 else 0
)
combined_df["LOAN_COUNT"] = combined_df.groupby("SK_ID_CURR")["SK_ID"].transform(
    "count"
)

# 3. current_id별 연체 횟수 / 전체 대출 횟수 = 연체 비율 칼럼
combined_df["OVERDUE_RATIO"] = (
    combined_df.groupby("SK_ID_CURR")["IS_OVERDUE"].transform("sum")
    / combined_df["LOAN_COUNT"]
)

# 4. 연체 횟수 총 cnt: LOAN_STATUS가 1인 경우 current_id로 그룹바이해서 카운트
combined_df["TOTAL_OVERDUE_COUNT"] = combined_df.groupby("SK_ID_CURR")[
    "IS_OVERDUE"
].transform("sum")

# 5. current_id별 평균 연체 횟수 칼럼
combined_df["AVERAGE_OVERDUE"] = (
    combined_df["TOTAL_OVERDUE_COUNT"] / combined_df["LOAN_COUNT"]
)

combined_df["HAS_OVERDUE"] = combined_df.groupby("SK_ID_CURR")["IS_OVERDUE"].transform(
    lambda x: 1 if x.sum() > 0 else 0
)

In [ ]:
combined_df.drop(
    ["AMT_ANNUITY", "AMT_CREDIT", "LOAN_STATUS", "SK_ID", "IS_OVERDUE", "IS_PAIDOFF"],
    axis=1,
    inplace=True,
)
combined_df.drop_duplicates(subset="SK_ID_CURR", inplace=True)
combined_df.dropna(inplace=True)  # 대출 월 납부액 NaN은 어쩔 수가 없다

In [ ]:
combined_df.info()

In [ ]:
combined_df.to_csv(path.join(DATA_PATH, "combined_df.csv"), index=False)

# application_train


In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
print(application_train.isnull().sum())

In [ ]:
# 데이터프레임 복사
df = application_train.copy()

# 1. 각 열별로 NaN 값의 개수를 세고, 10만 개 이상의 NaN 값을 가진 열을 제거
df.dropna(axis=1, thresh=len(df) - 100000, inplace=True)
print("NaN이 10만 개 이상인 열 제거 후:", df.shape)

# 2. 고유값이 10개 이상인 명목형 칼럼 제거
categorical_columns = df.select_dtypes(include=["object"]).columns
columns_to_drop = (
    df[categorical_columns].nunique()[df[categorical_columns].nunique() >= 10].index
)
df.drop(columns=columns_to_drop, inplace=True)
print("고유값 10개 이상인 명목형 칼럼 제거 후:", df.shape)

# 3. NaN 값이 있는 행 제거
df.dropna(inplace=True)
print("NaN 값이 있는 행 제거 후:", df.shape)

In [ ]:
# 먼저 combined_df를 df에 조인
merged_df = df.join(
    combined_df.set_index("SK_ID_CURR"),
    on="SK_ID_CURR",
    rsuffix="_combined",
    how="left",
)

# bureau를 조인
merged_df = merged_df.join(
    tri_bureau.set_index("SK_ID_CURR"), on="SK_ID_CURR", rsuffix="_bureau", how="left"
)

# previous_application을 조인
merged_df = merged_df.join(
    tri_previous_application.set_index("SK_ID_CURR"),
    on="SK_ID_CURR",
    rsuffix="_previous",
    how="left",
)

# 결과 확인
print(merged_df.shape)

In [ ]:
print(merged_df.isnull().sum())

In [ ]:
# 공통으로 존재하는 SK_ID_CURR 값의 개수를 확인
common_ids = set(df["SK_ID_CURR"]).intersection(
    set(combined_df["SK_ID_CURR"]),
    set(tri_bureau["SK_ID_CURR"]),
    set(tri_previous_application["SK_ID_CURR"]),
)

print(f"공통으로 존재하는 SK_ID_CURR 값의 수: {len(common_ids)}")
print(f"df에만 있는 SK_ID_CURR의 수: {len(set(df['SK_ID_CURR']) - common_ids)}")
print(
    f"combined_df에만 있는 SK_ID_CURR의 수: {len(set(combined_df['SK_ID_CURR']) - common_ids)}"
)
print(
    f"tri_bureau에만 있는 SK_ID_CURR의 수: {len(set(tri_bureau['SK_ID_CURR']) - common_ids)}"
)
print(
    f"tri_previous_application에만 있는 SK_ID_CURR의 수: {len(set(tri_previous_application['SK_ID_CURR']) - common_ids)}"
)

In [ ]:
merged_df.fillna(
    0, inplace=True
)  # 과거 대출기록이 없는 친구들 이므로 join해서 나온 NaN값은 전부 0으로 돌릴 수 있도록 변수 생성했음

### 최종본 파생변수


In [ ]:
merged_df["Dependents_Index"] = merged_df["CNT_CHILDREN"] / (
    merged_df["CNT_FAM_MEMBERS"] - merged_df["CNT_CHILDREN"]
)  # 부양 부담 지수
merged_df["Income_to_Dependents_Ratio"] = merged_df["AMT_INCOME_TOTAL"] / (
    merged_df["CNT_CHILDREN"] + 1
)  # 소득 대비 부양 부담 비율
merged_df["Debt_to_Income_Ratio"] = (
    merged_df["AMT_CREDIT_SUM_DEBT_MEAN"] / merged_df["AMT_INCOME_TOTAL"]
)  # 부채 상환 비율
merged_df["Debt_Repayment_Capability_Index"] = (
    merged_df["AMT_ANNUITY_MEAN"] / merged_df["AMT_INCOME_TOTAL"]
)  # 부채 상환 가능성 지수
merged_df["Credit_Utilization_Ratio"] = np.where(
    merged_df["AMT_CREDIT_SUM"] == 0,
    0,
    merged_df["AMT_CREDIT_SUM_DEBT_SUM"] / merged_df["AMT_CREDIT_SUM"],
)  # 대출 상환 비율

In [ ]:
merged_df.shape

In [ ]:
SAVE_PATH = ".tmp/dataset"
if not path.isdir(SAVE_PATH):
    os.makedirs(SAVE_PATH, exist_ok=True)

# 최종 데이터셋 : 학습 + 데모
merged_df.to_csv(path.join(SAVE_PATH, "dataset_total.csv"), index=False)

In [ ]:
merged_df.select_dtypes(include="object").head(10)